<a href="https://colab.research.google.com/github/rka0285058-afk/Earthquake_App/blob/main/Final_RKAWASTHI_App.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import pandas as pd

# Define the base URL for the USGS Earthquake API
base_url = 'https://earthquake.usgs.gov/fdsnws/event/1/query'

# Define parameters for the API request
params = {
    'format': 'geojson',
    'starttime': '2023-01-01',
    'endtime': '2023-01-31',
    'minmagnitude': 4.5,
    'latitude': 34.0522,
    'longitude': -118.2437,
    'maxradiuskm': 500
}

# Make the HTTP GET request
response = requests.get(base_url, params=params)

# Check if the request was successful
response.raise_for_status() # Raises an HTTPError for bad responses (4xx or 5xx)

# Parse the JSON response
earthquake_data = response.json()

print(f"Successfully fetched {len(earthquake_data['features'])} earthquake records.")
print("Keys in the earthquake data (top level):")
print(earthquake_data.keys())


In [ ]:
earthquake_features = earthquake_data['features']

# Extract relevant properties from each feature
earthquakes_list = []
for feature in earthquake_features:
    properties = feature['properties']
    geometry = feature['geometry']

    # Handle cases where geometry or coordinates might be missing
    latitude = geometry['coordinates'][1] if geometry and 'coordinates' in geometry and len(geometry['coordinates']) > 1 else None
    longitude = geometry['coordinates'][0] if geometry and 'coordinates' in geometry and len(geometry['coordinates']) > 0 else None

    # Add location data to properties
    properties['latitude'] = latitude
    properties['longitude'] = longitude

    # Add the full geometry type and coordinates if needed, or simplify
    properties['geometry_type'] = geometry['type'] if geometry else None

    earthquakes_list.append(properties)

# Create a pandas DataFrame from the list of earthquake properties
earthquake_df = pd.DataFrame(earthquakes_list)

# Display the first few rows of the DataFrame and its info to verify
print("Earthquake DataFrame Head:")
print(earthquake_df.head())
print("\nEarthquake DataFrame Info:")
print(earthquake_df.info())

In [ ]:
import requests
import pandas as pd

# Define the base URL for the USGS Earthquake API
base_url = 'https://earthquake.usgs.gov/fdsnws/event/1/query'

# Define parameters for the API request with broader scope
params = {
    'format': 'geojson',
    'starttime': '2023-01-01',
    'endtime': '2023-01-31',
    'minmagnitude': 4.5
    # Removed latitude, longitude, and maxradiuskm to get global data
}

# Make the HTTP GET request
response = requests.get(base_url, params=params)

# Check if the request was successful
response.raise_for_status() # Raises an HTTPError for bad responses (4xx or 5xx)

# Parse the JSON response
earthquake_data = response.json()

print(f"Successfully fetched {len(earthquake_data['features'])} earthquake records.")
print("Keys in the earthquake data (top level):")
print(earthquake_data.keys())

In [ ]:
earthquake_features = earthquake_data['features']

# Extract relevant properties from each feature
earthquakes_list = []
for feature in earthquake_features:
    properties = feature['properties']
    geometry = feature['geometry']

    # Handle cases where geometry or coordinates might be missing
    latitude = geometry['coordinates'][1] if geometry and 'coordinates' in geometry and len(geometry['coordinates']) > 1 else None
    longitude = geometry['coordinates'][0] if geometry and 'coordinates' in geometry and len(geometry['coordinates']) > 0 else None

    # Add location data to properties
    properties['latitude'] = latitude
    properties['longitude'] = longitude

    # Add the full geometry type and coordinates if needed, or simplify
    properties['geometry_type'] = geometry['type'] if geometry else None

    earthquakes_list.append(properties)

# Create a pandas DataFrame from the list of earthquake properties
earthquake_df = pd.DataFrame(earthquakes_list)

# Display the first few rows of the DataFrame and its info to verify
print("Earthquake DataFrame Head:")
print(earthquake_df.head())
print("\nEarthquake DataFrame Info:")
print(earthquake_df.info())

In [ ]:
import pandas as pd

# 1. Add a new column named 'depth' to the earthquake_df DataFrame
# The depth information is available as the third element (index 2) in the geometry['coordinates'] list.
# We can extract this from the original earthquake_data['features'] and align it with the DataFrame.

depths = []
for feature in earthquake_data['features']:
    if 'geometry' in feature and 'coordinates' in feature['geometry'] and len(feature['geometry']['coordinates']) > 2:
        depths.append(feature['geometry']['coordinates'][2])
    else:
        depths.append(None) # Append None if depth data is missing

earthquake_df['depth'] = depths

# 2. Convert the 'time' and 'updated' columns to readable datetime objects
earthquake_df['time'] = pd.to_datetime(earthquake_df['time'] / 1000, unit='s')
earthquake_df['updated'] = pd.to_datetime(earthquake_df['updated'] / 1000, unit='s')

# 3. Drop the 'tz' column as it contains only null values
# Check if 'tz' column exists before dropping to prevent error if it was already dropped or not present
if 'tz' in earthquake_df.columns:
    earthquake_df = earthquake_df.drop('tz', axis=1)

# 4. Display the first few rows of the updated DataFrame and its information to verify changes
print("Earthquake DataFrame Head after processing:")
print(earthquake_df.head())
print("\nEarthquake DataFrame Info after processing:")
print(earthquake_df.info())


In [ ]:
import folium
from branca.colormap import linear # Corrected import from branca.colormap

# Create a base map centered globally
m = folium.Map(location=[0, 0], zoom_start=2)

# Prepare for depth-based coloring. Find min and max depth for the colormap.
min_depth = earthquake_df['depth'].min()
max_depth = earthquake_df['depth'].max()

# Create a colormap for depth
depth_colormap = linear.YlOrRd_09.scale(min_depth, max_depth)

print(f"Base map created. Min depth: {min_depth:.2f} km, Max depth: {max_depth:.2f} km")

In [ ]:
for index, row in earthquake_df.iterrows():
    # Scale magnitude for marker radius (adjust factor as needed for visibility)
    # A base radius of 1.5-2 plus magnitude can work well, or a direct multiplication.
    # Using a slightly adjusted magnitude to avoid very small or excessively large markers
    radius = row['mag'] * 3 # Adjust multiplier for better visual scaling

    # Get color based on depth
    # Handle potential None or NaN depths by providing a default color (e.g., black or gray)
    depth_color = depth_colormap(row['depth']) if pd.notna(row['depth']) else '#000000' # Black for missing depth

    # Create popup content with detailed information
    popup_html = (
        f"<b>Magnitude:</b> {row['mag']}<br>" +
        f"<b>Location:</b> {row['place']}<br>" +
        f"<b>Time:</b> {row['time'].strftime('%Y-%m-%d %H:%M:%S')}<br>" +
        f"<b>Depth:</b> {row['depth']:.2f} km"
    )

    folium.CircleMarker(
        location=[row['latitude'], row['longitude']],
        radius=radius,
        color=depth_color,
        fill=True,
        fill_color=depth_color,
        fill_opacity=0.7,
        tooltip=popup_html, # Tooltip shows on hover
        popup=popup_html    # Popup shows on click
    ).add_to(m)

print("Added earthquake markers to the map.")

In [ ]:
depth_colormap.add_to(m)
m.save('earthquake_map.html') # Save the map to an HTML file

print("Map saved to 'earthquake_map.html'. Open this file in a web browser to view the interactive map.")
# To display the map directly in a Jupyter environment, you can simply call 'm'
m

In [ ]:
import pandas as pd

# Set display options to show all columns and rows for better inspection
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# Display the full earthquake_df DataFrame
print("Full Earthquake DataFrame:")
print(earthquake_df)

# Identify and display a subset of the DataFrame with key attributes
relevant_columns = ['time', 'place', 'mag', 'depth', 'latitude', 'longitude']
earthquake_overview_df = earthquake_df[relevant_columns]

print("\nEarthquake Overview (Key Attributes):")
print(earthquake_overview_df)

In [ ]:
import requests
import pandas as pd

# Define the base URL for the USGS Earthquake API
base_url = 'https://earthquake.usgs.gov/fdsnws/event/1/query'

# Define parameters for the API request with a more constrained date range
params = {
    'format': 'geojson',
    'starttime': '2022-01-01', # Adjusted starttime to a more recent date for a manageable query
    'endtime': '2023-01-31',
    'minmagnitude': 4.5
    # Removed latitude, longitude, and maxradiuskm to get global data
}

# Make the HTTP GET request
response = requests.get(base_url, params=params)

# Check if the request was successful
response.raise_for_status() # Raises an HTTPError for bad responses (4xx or 5xx)

# Parse the JSON response
earthquake_data = response.json()

print(f"Successfully fetched {len(earthquake_data['features'])} earthquake records.")
print("Keys in the earthquake data (top level):")
print(earthquake_data.keys())

In [ ]:
earthquake_features = earthquake_data['features']

# Extract relevant properties from each feature
earthquakes_list = []
for feature in earthquake_features:
    properties = feature['properties']
    geometry = feature['geometry']

    # Handle cases where geometry or coordinates might be missing
    latitude = geometry['coordinates'][1] if geometry and 'coordinates' in geometry and len(geometry['coordinates']) > 1 else None
    longitude = geometry['coordinates'][0] if geometry and 'coordinates' in geometry and len(geometry['coordinates']) > 0 else None

    # Add location data to properties
    properties['latitude'] = latitude
    properties['longitude'] = longitude

    # Add the full geometry type and coordinates if needed, or simplify
    properties['geometry_type'] = geometry['type'] if geometry else None

    earthquakes_list.append(properties)

# Create a pandas DataFrame from the list of earthquake properties
earthquake_df = pd.DataFrame(earthquakes_list)

# Display the first few rows of the DataFrame and its info to verify
print("Earthquake DataFrame Head:")
print(earthquake_df.head())
print("\nEarthquake DataFrame Info:")
print(earthquake_df.info())

In [ ]:
import pandas as pd

# 1. Add a new column named 'depth' to the earthquake_df DataFrame
# The depth information is available as the third element (index 2) in the geometry['coordinates'] list.
# We can extract this from the original earthquake_data['features'] and align it with the DataFrame.

depths = []
for feature in earthquake_data['features']:
    if 'geometry' in feature and 'coordinates' in feature['geometry'] and len(feature['geometry']['coordinates']) > 2:
        depths.append(feature['geometry']['coordinates'][2])
    else:
        depths.append(None) # Append None if depth data is missing

earthquake_df['depth'] = depths

# 2. Convert the 'time' and 'updated' columns to readable datetime objects
earthquake_df['time'] = pd.to_datetime(earthquake_df['time'] / 1000, unit='s')
earthquake_df['updated'] = pd.to_datetime(earthquake_df['updated'] / 1000, unit='s')

# 3. Drop the 'tz' column as it contains only null values
# Check if 'tz' column exists before dropping to prevent error if it was already dropped or not present
if 'tz' in earthquake_df.columns:
    earthquake_df = earthquake_df.drop('tz', axis=1)

# 4. Display the first few rows of the updated DataFrame and its information to verify changes
print("Earthquake DataFrame Head after processing:")
print(earthquake_df.head())
print("\nEarthquake DataFrame Info after processing:")
print(earthquake_df.info())

In [ ]:
import folium
from branca.colormap import linear

# Create a base map centered globally
m = folium.Map(location=[0, 0], zoom_start=2)

# Prepare for depth-based coloring. Find min and max depth for the colormap.
min_depth = earthquake_df['depth'].min()
max_depth = earthquake_df['depth'].max()

# Create a colormap for depth
depth_colormap = linear.YlOrRd_09.scale(min_depth, max_depth)

print(f"Base map created. Min depth: {min_depth:.2f} km, Max depth: {max_depth:.2f} km")

In [ ]:
for index, row in earthquake_df.iterrows():
    # 2. Calculate the marker radius by multiplying the 'mag' (magnitude) column by a scaling factor.
    radius = row['mag'] * 3  # Adjust multiplier for better visual scaling

    # 3. Determine the marker color based on the 'depth' column, using the `depth_colormap`.
    # Handle potential None or NaN depths by providing a default color (e.g., black or gray)
    depth_color = depth_colormap(row['depth']) if pd.notna(row['depth']) else '#000000' # Black for missing depth

    # 4. Create an HTML string for the tooltip and popup content.
    popup_html = (
        f"<b>Magnitude:</b> {row['mag']}<br>" +
        f"<b>Location:</b> {row['place']}<br>" +
        f"<b>Time:</b> {row['time'].strftime('%Y-%m-%d %H:%M:%S')}<br>" +
        f"<b>Depth:</b> {row['depth']:.2f} km"
    )

    # 5, 6, 7. Create a folium.CircleMarker object with specified properties.
    folium.CircleMarker(
        location=[row['latitude'], row['longitude']],
        radius=radius,
        color=depth_color,
        fill=True,
        fill_color=depth_color,
        fill_opacity=0.7,
        tooltip=popup_html,  # Tooltip shows on hover
        popup=popup_html     # Popup shows on click
    ).add_to(m) # 8. Add the CircleMarker to the map object m.

print("Added earthquake markers to the map.")

In [ ]:
depth_colormap.add_to(m)
m.save('earthquake_map.html') # Save the map to an HTML file

print("Map saved to 'earthquake_map.html'. Open this file in a web browser to view the interactive map.")
# To display the map directly in a Jupyter environment, you can simply call 'm'
m

In [ ]:
import requests

# 1. Define the URL for the GeoJSON file containing tectonic plate boundaries
tectonic_plates_url = 'https://raw.githubusercontent.com/fraxen/tectonicplates/master/GeoJSON/PB2002_boundaries.json'

# 2. Use the requests library to make an HTTP GET request
response = requests.get(tectonic_plates_url)

# 3. Check if the request was successful
response.raise_for_status() # Raises an HTTPError for bad responses (4xx or 5xx)

# 4. Parse the JSON response into a Python dictionary
tectonic_plates_geojson = response.json()

print("Successfully fetched tectonic plate GeoJSON data.")
print(f"Keys in tectonic_plates_geojson: {tectonic_plates_geojson.keys()}")

In [ ]:
import json

# 1. Initialize an empty list to store GeoJSON features
features = []

# 2. Iterate through each row of the earthquake_df DataFrame
for index, row in earthquake_df.iterrows():
    # 3a. Define the geometry of the GeoJSON feature
    geometry = {
        'type': 'Point',
        'coordinates': [row['longitude'], row['latitude']]
    }

    # 3b. Prepare the properties for the GeoJSON feature
    # Convert time to ISO format string as required for TimestampedGeoJson
    properties = {
        'time': row['time'].isoformat(),
        'magnitude': row['mag'],
        'place': row['place'],
        'depth': round(row['depth'], 2) if pd.notna(row['depth']) else None,
        'popup': (
            f"<b>Magnitude:</b> {row['mag']}<br>" +
            f"<b>Location:</b> {row['place']}<br>" +
            f"<b>Time:</b> {row['time'].strftime('%Y-%m-%d %H:%M:%S')}<br>" +
            f"<b>Depth:</b> {row['depth']:.2f} km"
        )
    }

    # 3c. Create a GeoJSON Feature dictionary
    feature = {
        'type': 'Feature',
        'geometry': geometry,
        'properties': properties
    }

    # 3d. Append this Feature dictionary to the features list
    features.append(feature)

# 4. Construct the final GeoJSON earthquakes_geojson dictionary
earthquakes_geojson = {
    'type': 'FeatureCollection',
    'features': features
}

# 5. Print the first few entries of the features list and the overall earthquakes_geojson keys to verify the structure
print("First 2 GeoJSON features:")
for i in range(min(2, len(features))):
    print(json.dumps(features[i], indent=2))

print(f"\nTotal features in GeoJSON: {len(earthquakes_geojson['features'])}")
print(f"Keys in earthquakes_geojson: {earthquakes_geojson.keys()}")

In [ ]:
import folium
from branca.colormap import linear

# Ensure the map 'm' is initialized (re-initialize if running this block independently)
# If 'm' already exists from previous steps, this will re-create it.
m = folium.Map(location=[0, 0], zoom_start=2)

# Add tectonic plate boundaries as a GeoJSON layer
folium.GeoJson(
    tectonic_plates_geojson,
    name='Tectonic Plates',
    style_function=lambda x: {'color': 'orange', 'weight': 2},
    tooltip=folium.GeoJsonTooltip(fields=['Name'], aliases=['Plate Name:'], localize=True),
    control=True
).add_to(m)

# Define magnitude ranges and corresponding FeatureGroups
mag_4_5_5_5 = folium.FeatureGroup(name='Magnitude 4.5-5.5')
mag_5_5_6_5 = folium.FeatureGroup(name='Magnitude 5.5-6.5')
mag_gt_6_5 = folium.FeatureGroup(name='Magnitude > 6.5')

# Prepare for depth-based coloring. Find min and max depth for the colormap.
min_depth = earthquake_df['depth'].min()
max_depth = earthquake_df['depth'].max()

# Create a colormap for depth
depth_colormap = linear.YlOrRd_09.scale(min_depth, max_depth)

print("Base map initialized with tectonic plates layer.")
print(f"Magnitude FeatureGroups created. Min depth: {min_depth:.2f} km, Max depth: {max_depth:.2f} km")

In [ ]:
for index, row in earthquake_df.iterrows():
    # Calculate the marker radius by multiplying the 'mag' (magnitude) column by a scaling factor.
    radius = row['mag'] * 3  # Adjust multiplier for better visual scaling

    # Determine the marker color based on the 'depth' column, using the `depth_colormap`.
    # Handle potential None or NaN depths by providing a default color (e.g., black or gray)
    depth_color = depth_colormap(row['depth']) if pd.notna(row['depth']) else '#000000' # Black for missing depth

    # Create an HTML string for the tooltip and popup content.
    popup_html = (
        f"<b>Magnitude:</b> {row['mag']}<br>" +
        f"<b>Location:</b> {row['place']}<br>" +
        f"<b>Time:</b> {row['time'].strftime('%Y-%m-%d %H:%M:%S')}<br>" +
        f"<b>Depth:</b> {row['depth']:.2f} km"
    )

    # Create a folium.CircleMarker object with specified properties.
    marker = folium.CircleMarker(
        location=[row['latitude'], row['longitude']],
        radius=radius,
        color=depth_color,
        fill=True,
        fill_color=depth_color,
        fill_opacity=0.7,
        tooltip=popup_html,  # Tooltip shows on hover
        popup=popup_html     # Popup shows on click
    )

    # Assign markers to appropriate FeatureGroup based on magnitude
    if 4.5 <= row['mag'] < 5.5:
        marker.add_to(mag_4_5_5_5)
    elif 5.5 <= row['mag'] < 6.5:
        marker.add_to(mag_5_5_6_5)
    elif row['mag'] >= 6.5:
        marker.add_to(mag_gt_6_5)

# Add the FeatureGroups to the map
mag_4_5_5_5.add_to(m)
mag_5_5_6_5.add_to(m)
mag_gt_6_5.add_to(m)

print("Added earthquake markers to magnitude-based FeatureGroups and added them to the map.")

In [ ]:
depth_colormap.caption = 'Earthquake Depth (km)'
depth_colormap.add_to(m)

# Add LayerControl to the map to allow toggling of FeatureGroups
folium.LayerControl().add_to(m)

m.save('earthquake_map_with_overlays.html') # Save the map to an HTML file

print("Map with tectonic plates, magnitude-based earthquake layers, and depth colormap saved to 'earthquake_map_with_overlays.html'.")
print("Open this file in a web browser to view the interactive map with layer control.")
# To display the map directly in a Jupyter environment, you can simply call 'm'
m

In [ ]:
from folium.plugins import TimestampedGeoJson

# Add the TimestampedGeoJson layer for animation
TimestampedGeoJson(
    earthquakes_geojson,
    period='P1D', # Aggregate data daily for animation
    auto_play=True,
    loop=False,
    transition_time=200, # Milliseconds for transition between frames
    date_options='YYYY/MM/DD'
).add_to(m)

print("Timestamped GeoJSON layer for earthquake animation added to the map.")

In [ ]:
m.save('earthquake_map_animated.html') # Save the map to an HTML file including the animation

print("Map with animated earthquake layer saved to 'earthquake_map_animated.html'. Open this file in a web browser to view the interactive map.")
# To display the map directly in a Jupyter environment, you can simply call 'm'
m

In [ ]:
import json

# 1. Initialize an empty list to store GeoJSON features
features = []

# 2. Iterate through each row of the earthquake_df DataFrame
for index, row in earthquake_df.iterrows():
    # 3a. Define the geometry of the GeoJSON feature
    geometry = {
        'type': 'Point',
        'coordinates': [row['longitude'], row['latitude']]
    }

    # 3b. Prepare the properties for the GeoJSON feature
    # Convert time to ISO format string as required for TimestampedGeoJson
    properties = {
        'time': row['time'].isoformat(),
        'magnitude': row['mag'],
        'place': row['place'],
        'depth': round(row['depth'], 2) if pd.notna(row['depth']) else None,
        'popup': (
            f"<b>Magnitude:</b> {row['mag']}<br>" +
            f"<b>Location:</b> {row['place']}<br>" +
            f"<b>Time:</b> {row['time'].strftime('%Y-%m-%d %H:%M:%S')}<br>" +
            f"<b>Depth:</b> {row['depth']:.2f} km"
        )
    }

    # 3c. Create a GeoJSON Feature dictionary
    feature = {
        'type': 'Feature',
        'geometry': geometry,
        'properties': properties
    }

    # 3d. Append this Feature dictionary to the features list
    features.append(feature)

# 4. Construct the final GeoJSON earthquakes_geojson dictionary
earthquakes_geojson = {
    'type': 'FeatureCollection',
    'features': features
}

# 5. Print the first few entries of the features list and the overall earthquakes_geojson keys to verify the structure
print("First 2 GeoJSON features:")
for i in range(min(2, len(features))):
    print(json.dumps(features[i], indent=2))

print(f"\nTotal features in GeoJSON: {len(earthquakes_geojson['features'])}")
print(f"Keys in earthquakes_geojson: {earthquakes_geojson.keys()}")

In [ ]:
import requests
import pandas as pd

def fetch_earthquake_data(starttime, endtime, minmagnitude):
    """Fetches earthquake data from the USGS API for a specified date range and minimum magnitude."""
    base_url = 'https://earthquake.usgs.gov/fdsnws/event/1/query'
    params = {
        'format': 'geojson',
        'starttime': starttime,
        'endtime': endtime,
        'minmagnitude': minmagnitude
    }
    response = requests.get(base_url, params=params)
    response.raise_for_status() # Raises an HTTPError for bad responses (4xx or 5xx)
    return response.json()

def process_earthquake_data(earthquake_data):
    """Processes raw earthquake JSON data into a cleaned pandas DataFrame."""
    earthquake_features = earthquake_data['features']
    earthquakes_list = []

    for feature in earthquake_features:
        properties = feature['properties']
        geometry = feature['geometry']

        latitude = geometry['coordinates'][1] if geometry and 'coordinates' in geometry and len(geometry['coordinates']) > 1 else None
        longitude = geometry['coordinates'][0] if geometry and 'coordinates' in geometry and len(geometry['coordinates']) > 0 else None
        depth = geometry['coordinates'][2] if geometry and 'coordinates' in geometry and len(geometry['coordinates']) > 2 else None

        properties['latitude'] = latitude
        properties['longitude'] = longitude
        properties['geometry_type'] = geometry['type'] if geometry else None
        properties['depth'] = depth # Add depth directly from geometry coordinates

        earthquakes_list.append(properties)

    earthquake_df = pd.DataFrame(earthquakes_list)

    # Convert time and updated columns to datetime objects
    if 'time' in earthquake_df.columns:
        earthquake_df['time'] = pd.to_datetime(earthquake_df['time'] / 1000, unit='s')
    if 'updated' in earthquake_df.columns:
        earthquake_df['updated'] = pd.to_datetime(earthquake_df['updated'] / 1000, unit='s')

    # Drop 'tz' column if it exists and contains mostly null values
    if 'tz' in earthquake_df.columns and earthquake_df['tz'].isnull().mean() > 0.5: # Check if more than half are null
        earthquake_df = earthquake_df.drop('tz', axis=1)

    return earthquake_df

# Call the functions to fetch and process data
raw_earthquake_data = fetch_earthquake_data(starttime='2022-01-01', endtime='2023-01-31', minmagnitude=4.5)
earthquake_df = process_earthquake_data(raw_earthquake_data)

# Display the first few rows and info of the processed DataFrame
print("Earthquake DataFrame Head after refactoring:")
print(earthquake_df.head())
print("\nEarthquake DataFrame Info after refactoring:")
print(earthquake_df.info())

In [ ]:
import folium
from branca.colormap import linear
import requests
import json
from folium.plugins import TimestampedGeoJson

def create_base_map_and_colormap(earthquake_df):
    """Initializes a folium.Map and creates a colormap based on earthquake depths."""
    m = folium.Map(location=[0, 0], zoom_start=2)

    min_depth = earthquake_df['depth'].min()
    max_depth = earthquake_df['depth'].max()
    depth_colormap = linear.YlOrRd_09.scale(min_depth, max_depth)
    return m, depth_colormap

def add_tectonic_plates(m, tectonic_plates_geojson):
    """Adds tectonic plate boundaries as a GeoJSON layer to the map."""
    folium.GeoJson(
        tectonic_plates_geojson,
        name='Tectonic Plates',
        style_function=lambda x: {'color': 'orange', 'weight': 2},
        tooltip=folium.GeoJsonTooltip(fields=['Name'], aliases=['Plate Name:'], localize=True),
        control=True
    ).add_to(m)

def add_earthquake_layers(m, earthquake_df, depth_colormap):
    """Adds earthquake markers to the map, categorized by magnitude into FeatureGroups."""
    mag_4_5_5_5 = folium.FeatureGroup(name='Magnitude 4.5-5.5')
    mag_5_5_6_5 = folium.FeatureGroup(name='Magnitude 5.5-6.5')
    mag_gt_6_5 = folium.FeatureGroup(name='Magnitude > 6.5')

    for index, row in earthquake_df.iterrows():
        radius = row['mag'] * 3
        depth_color = depth_colormap(row['depth']) if pd.notna(row['depth']) else '#000000'

        popup_html = (
            f"<b>Magnitude:</b> {row['mag']}<br>" +
            f"<b>Location:</b> {row['place']}<br>" +
            f"<b>Time:</b> {row['time'].strftime('%Y-%m-%d %H:%M:%S')}<br>" +
            f"<b>Depth:</b> {row['depth']:.2f} km"
        )

        marker = folium.CircleMarker(
            location=[row['latitude'], row['longitude']],
            radius=radius,
            color=depth_color,
            fill=True,
            fill_color=depth_color,
            fill_opacity=0.7,
            tooltip=popup_html,
            popup=popup_html
        )

        if 4.5 <= row['mag'] < 5.5:
            marker.add_to(mag_4_5_5_5)
        elif 5.5 <= row['mag'] < 6.5:
            marker.add_to(mag_5_5_6_5)
        elif row['mag'] >= 6.5:
            marker.add_to(mag_gt_6_5)

    mag_4_5_5_5.add_to(m)
    mag_5_5_6_5.add_to(m)
    mag_gt_6_5.add_to(m)

    depth_colormap.caption = 'Earthquake Depth (km)'
    depth_colormap.add_to(m)
    folium.LayerControl().add_to(m)

def prepare_geojson_for_timeslider(earthquake_df):
    """Transforms earthquake_df into a GeoJSON FeatureCollection for the time slider."""
    features = []
    for index, row in earthquake_df.iterrows():
        geometry = {
            'type': 'Point',
            'coordinates': [row['longitude'], row['latitude']]
        }
        properties = {
            'time': row['time'].isoformat(),
            'magnitude': row['mag'],
            'place': row['place'],
            'depth': round(row['depth'], 2) if pd.notna(row['depth']) else None,
            'popup': (
                f"<b>Magnitude:</b> {row['mag']}<br>" +
                f"<b>Location:</b> {row['place']}<br>" +
                f"<b>Time:</b> {row['time'].strftime('%Y-%m-%d %H:%M:%S')}<br>" +
                f"<b>Depth:</b> {row['depth']:.2f} km"
            )
        }
        feature = {
            'type': 'Feature',
            'geometry': geometry,
            'properties': properties
        }
        features.append(feature)
    return {'type': 'FeatureCollection', 'features': features}

def add_animated_timeslider(m, earthquakes_geojson):
    """Integrates a TimestampedGeoJson layer for animated earthquake visualization."""
    TimestampedGeoJson(
        earthquakes_geojson,
        period='P1D',
        auto_play=True,
        loop=False,
        transition_time=200,
        date_options='YYYY/MM/DD'
    ).add_to(m)

print("All refactoring functions defined successfully.")


In [ ]:
import requests

# 1. Fetch tectonic plates GeoJSON data (if not already loaded)
# This is duplicated from an earlier cell for reusability if this block is run independently
tectonic_plates_url = 'https://raw.githubusercontent.com/fraxen/tectonicplates/master/GeoJSON/PB2002_boundaries.json'
response = requests.get(tectonic_plates_url)
response.raise_for_status()
tectonic_plates_geojson = response.json()

# 2. Call functions sequentially to build the complete map
m, depth_colormap = create_base_map_and_colormap(earthquake_df)

add_tectonic_plates(m, tectonic_plates_geojson)

add_earthquake_layers(m, earthquake_df, depth_colormap)

earthquakes_geojson_for_slider = prepare_geojson_for_timeslider(earthquake_df)

add_animated_timeslider(m, earthquakes_geojson_for_slider)

# 3. Save the map to an HTML file and display it
m.save('earthquake_map_refactored_final.html')

print("Refactored map with tectonic plates, magnitude-based earthquake layers, depth colormap, and animated time slider saved to 'earthquake_map_refactored_final.html'.")
print("Open this file in a web browser to view the interactive map with layer control and animation.")
# To display the map directly in a Jupyter environment, you can simply call 'm'
m

In [ ]:
%%writefile streamlit_app.py
import streamlit as st
import pandas as pd
import requests
import folium
from branca.colormap import linear
from folium.plugins import TimestampedGeoJson
import json
import datetime

# Refactored Functions (copied from previous notebook cells to make the script self-contained)

def fetch_earthquake_data(starttime, endtime, minmagnitude):
    """Fetches earthquake data from the USGS API for a specified date range and minimum magnitude."""
    base_url = 'https://earthquake.usgs.gov/fdsnws/event/1/query'
    params = {
        'format': 'geojson',
        'starttime': starttime,
        'endtime': endtime,
        'minmagnitude': minmagnitude
    }
    response = requests.get(base_url, params=params)
    response.raise_for_status() # Raises an HTTPError for bad responses (4xx or 5xx)
    return response.json()

def process_earthquake_data(earthquake_data):
    """Processes raw earthquake JSON data into a cleaned pandas DataFrame."""
    earthquake_features = earthquake_data['features']
    earthquakes_list = []

    for feature in earthquake_features:
        properties = feature['properties']
        geometry = feature['geometry']

        latitude = geometry['coordinates'][1] if geometry and 'coordinates' in geometry and len(geometry['coordinates']) > 1 else None
        longitude = geometry['coordinates'][0] if geometry and 'coordinates' in geometry and len(geometry['coordinates']) > 0 else None
        depth = geometry['coordinates'][2] if geometry and 'coordinates' in geometry and len(geometry['coordinates']) > 2 else None

        properties['latitude'] = latitude
        properties['longitude'] = longitude
        properties['geometry_type'] = geometry['type'] if geometry else None
        properties['depth'] = depth # Add depth directly from geometry coordinates

        earthquakes_list.append(properties)

    earthquake_df = pd.DataFrame(earthquakes_list)

    # Convert time and updated columns to datetime objects
    if 'time' in earthquake_df.columns:
        earthquake_df['time'] = pd.to_datetime(earthquake_df['time'] / 1000, unit='s')
    if 'updated' in earthquake_df.columns:
        earthquake_df['updated'] = pd.to_datetime(earthquake_df['updated'] / 1000, unit='s')

    # Drop 'tz' column if it exists and contains mostly null values
    if 'tz' in earthquake_df.columns and earthquake_df['tz'].isnull().mean() > 0.5:
        earthquake_df = earthquake_df.drop('tz', axis=1)

    return earthquake_df

def create_base_map_and_colormap(earthquake_df):
    """Initializes a folium.Map and creates a colormap based on earthquake depths."""
    m = folium.Map(location=[0, 0], zoom_start=2)

    min_depth = earthquake_df['depth'].min()
    max_depth = earthquake_df['depth'].max()
    depth_colormap = linear.YlOrRd_09.scale(min_depth, max_depth)
    return m, depth_colormap

def add_tectonic_plates(m, tectonic_plates_geojson):
    """Adds tectonic plate boundaries as a GeoJSON layer to the map."""
    folium.GeoJson(
        tectonic_plates_geojson,
        name='Tectonic Plates',
        style_function=lambda x: {'color': 'orange', 'weight': 2},
        tooltip=folium.GeoJsonTooltip(fields=['Name'], aliases=['Plate Name:'], localize=True),
        control=True
    ).add_to(m)

def add_earthquake_layers(m, earthquake_df, depth_colormap):
    """Adds earthquake markers to the map, categorized by magnitude into FeatureGroups."""
    mag_4_5_5_5 = folium.FeatureGroup(name='Magnitude 4.5-5.5')
    mag_5_5_6_5 = folium.FeatureGroup(name='Magnitude 5.5-6.5')
    mag_gt_6_5 = folium.FeatureGroup(name='Magnitude > 6.5')

    for index, row in earthquake_df.iterrows():
        radius = row['mag'] * 3
        depth_color = depth_colormap(row['depth']) if pd.notna(row['depth']) else '#000000'

        popup_html = (
            f"<b>Magnitude:</b> {row['mag']}<br>" +
            f"<b>Location:</b> {row['place']}<br>" +
            f"<b>Time:</b> {row['time'].strftime('%Y-%m-%d %H:%M:%S')}<br>" +
            f"<b>Depth:</b> {row['depth']:.2f} km"
        )

        marker = folium.CircleMarker(
            location=[row['latitude'], row['longitude']],
            radius=radius,
            color=depth_color,
            fill=True,
            fill_color=depth_color,
            fill_opacity=0.7,
            tooltip=popup_html,
            popup=popup_html
        )

        if 4.5 <= row['mag'] < 5.5:
            marker.add_to(mag_4_5_5_5)
        elif 5.5 <= row['mag'] < 6.5:
            marker.add_to(mag_5_5_6_5)
        elif row['mag'] >= 6.5:
            marker.add_to(mag_gt_6_5)

    mag_4_5_5_5.add_to(m)
    mag_5_5_6_5.add_to(m)
    mag_gt_6_5.add_to(m)

    depth_colormap.caption = 'Earthquake Depth (km)'
    depth_colormap.add_to(m)
    folium.LayerControl().add_to(m)

def prepare_geojson_for_timeslider(earthquake_df):
    """Transforms earthquake_df into a GeoJSON FeatureCollection for the time slider."""
    features = []
    for index, row in earthquake_df.iterrows():
        geometry = {
            'type': 'Point',
            'coordinates': [row['longitude'], row['latitude']]
        }
        properties = {
            'time': row['time'].isoformat(),
            'magnitude': row['mag'],
            'place': row['place'],
            'depth': round(row['depth'], 2) if pd.notna(row['depth']) else None,
            'popup': (
                f"<b>Magnitude:</b> {row['mag']}<br>" +
                f"<b>Location:</b> {row['place']}<br>" +
                f"<b>Time:</b> {row['time'].strftime('%Y-%m-%d %H:%M:%S')}<br>" +
                f"<b>Depth:</b> {row['depth']:.2f} km"
            )
        }
        feature = {
            'type': 'Feature',
            'geometry': geometry,
            'properties': properties
        }
        features.append(feature)
    return {'type': 'FeatureCollection', 'features': features}

def add_animated_timeslider(m, earthquakes_geojson):
    """Integrates a TimestampedGeoJson layer for animated earthquake visualization."""
    TimestampedGeoJson(
        earthquakes_geojson,
        period='P1D',
        auto_play=True,
        loop=False,
        transition_time=200,
        date_options='YYYY/MM/DD'
    ).add_to(m)

# --- Streamlit App Layout ---

st.set_page_config(layout="wide", page_title="Global Earthquake Visualizer")

st.title("Global Earthquake Visualizer")

st.markdown("Fetch, process, and visualize earthquake data from the USGS Earthquake Hazards Program.")

# Sidebar for user inputs
st.sidebar.header("Filter Earthquake Data")

# Date Range Inputs
today = datetime.date.today()
start_date = st.sidebar.date_input("Start Date", value=today - datetime.timedelta(days=365))
end_date = st.sidebar.date_input("End Date", value=today)

# Magnitude Slider
min_magnitude = st.sidebar.slider(
    "Minimum Magnitude",
    min_value=2.5,
    max_value=9.0,
    value=4.5,
    step=0.1
)

# Button to trigger data fetch
if st.sidebar.button("Fetch and Visualize Earthquakes"):
    st.subheader("Earthquake Data")
    with st.spinner("Fetching and processing earthquake data..."):
        try:
            # Fetch and process data
            raw_data = fetch_earthquake_data(str(start_date), str(end_date), min_magnitude)
            earthquake_df = process_earthquake_data(raw_data)

            if not earthquake_df.empty:
                st.success(f"Successfully fetched {len(earthquake_df)} earthquake records.")
                st.dataframe(earthquake_df[['time', 'place', 'mag', 'depth', 'latitude', 'longitude']].head())

                # Store processed data in session state for reuse
                st.session_state['earthquake_df'] = earthquake_df

            else:
                st.warning("No earthquake records found for the selected criteria.")
                st.session_state['earthquake_df'] = pd.DataFrame()

        except requests.exceptions.HTTPError as e:
            st.error(f"HTTP Error: {e}. Please check your input parameters or API limits.")
            st.session_state['earthquake_df'] = pd.DataFrame()
        except Exception as e:
            st.error(f"An error occurred: {e}")
            st.session_state['earthquake_df'] = pd.DataFrame()

# Display map if data exists in session state
if 'earthquake_df' in st.session_state and not st.session_state['earthquake_df'].empty:
    st.subheader("Interactive Earthquake Map")
    earthquake_df = st.session_state['earthquake_df']

    # Fetch tectonic plates GeoJSON data
    tectonic_plates_url = 'https://raw.githubusercontent.com/fraxen/tectonicplates/master/GeoJSON/PB2002_boundaries.json'
    try:
        response = requests.get(tectonic_plates_url)
        response.raise_for_status()
        tectonic_plates_geojson = response.json()
    except Exception as e:
        st.error(f"Could not fetch tectonic plate data: {e}")
        tectonic_plates_geojson = {'type': 'FeatureCollection', 'features': []}

    # Build the map using refactored functions
    m, depth_colormap = create_base_map_and_colormap(earthquake_df)
    add_tectonic_plates(m, tectonic_plates_geojson)
    add_earthquake_layers(m, earthquake_df, depth_colormap)
    earthquakes_geojson_for_slider = prepare_geojson_for_timeslider(earthquake_df)
    add_animated_timeslider(m, earthquakes_geojson_for_slider)

    # Display the map using streamlit_folium
    from streamlit_folium import folium_static
    folium_static(m)

    st.markdown("**Map Legend:**")
    st.markdown("\u2022 **Marker Size:** Proportional to Magnitude")
    st.markdown("\u2022 **Marker Color:** Indicates Depth (Redder = Shallower, Yellower = Deeper)")

    st.subheader("Detailed Earthquake Information")
    st.dataframe(earthquake_df[['time', 'place', 'mag', 'depth', 'latitude', 'longitude']])



In [ ]:
!streamlit run streamlit_app.py